In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ITESO-Kmeans").getOrCreate()

file_path = '/content/wine-clustering.csv'
wine_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [ ]:
from pyspark.ml.feature import VectorAssembler

wine_df_without_label = wine_df.drop("label")
assembler = VectorAssembler(inputCols=wine_df_without_label.columns, outputCol="features")
wine_data = assembler.transform(wine_df).select("features")

In [ ]:

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

k_values = [2, 10, 15, 20]
df_scores = {}

for k in k_values:
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(wine_data)

    predictions = model.transform(wine_data)

    evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette")
    silhouette = evaluator.evaluate(predictions)
    df_scores[k] = silhouette
    print(f"Silhouette score for k={k}: {silhouette}")

print("Final S Scores:", df_scores)